In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import sys
module_path = '/kaggle/input/bratsslmsamodel'  # Update this with the correct path
sys.path.append(module_path)

# Import your module
import bratsslmsa


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/name_mapping_validation_data.csv
/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/survival_evaluation.csv
/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_084/BraTS20_Validation_084_flair.nii
/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_084/BraTS20_Validation_084_t2.nii
/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_084/BraTS20_Validation_084_t1ce.nii
/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_084/BraTS20_Validation_084_t1.nii
/kaggle/input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_Br

In [11]:
import numpy as np
import nibabel as nib
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.model_selection import train_test_split
import glob

# Define a function to load and preprocess a single volume
def load_and_preprocess_volume(volume_path, mask_path):
    
    # Load the volume and mask for each channel
    t2_volume = nib.load(volume_path[0]).get_fdata()
    t2_volume_scaled = scaler.fit_transform(t2_volume.reshape(-1, t2_volume.shape[-1])).reshape(t2_volume.shape)
    t1ce_volume = nib.load(volume_path[1]).get_fdata() 
    t1ce_volume_scaled = scaler.fit_transform(t1ce_volume.reshape(-1, t1ce_volume.shape[-1])).reshape(t1ce_volume.shape)
    flair_volume = nib.load(volume_path[2]).get_fdata()
    flair_volume_scaled = scaler.fit_transform(flair_volume.reshape(-1, flair_volume.shape[-1])).reshape(flair_volume.shape)
    mask = nib.load(mask_path).get_fdata().astype(np.uint8)
    mask[mask == 4] = 3  # Reassign mask values 4 to 3 or perform any other necessary preprocessing

    # Combine the channels
    combined_x = np.stack([t2_volume_scaled, t1ce_volume_scaled, flair_volume_scaled], axis=-1)
    return combined_x, mask

# Define a data generator
def custom_datagen(volume_path, mask_paths, batch_size):
    
    num_samples = len(volume_path)
    print("volume_path len:",len(volume_path))
    while True:
        for i in range(0, num_samples, batch_size):
            batch_volume_paths = volume_path[i:i + batch_size]
            batch_mask_paths = mask_paths[i:i + batch_size]
            X = []
            Y = []
            for volume_path1, mask_path in zip(batch_volume_paths, batch_mask_paths):
                             
                image, mask = load_and_preprocess_volume(volume_path1, mask_path)

                # Crop to a size divisible by the patch size
                image = image[56:184, 56:184, 13:141]
                mask = mask[56:184, 56:184, 13:141]
                temp_mask= to_categorical(mask, num_classes=4)
               
                X.append(image)
                Y.append(temp_mask)
            
            X = np.array(X)
            Y = np.array(Y)
            
            yield X, Y

# Define the data directory
data_directory = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'

# Create lists of file paths
t2_list = sorted(glob.glob(os.path.join(data_directory,'*/','*_t2.nii'), recursive=True))
remove = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_t2.nii' 
t2_list = [item for item in t2_list if item != remove]
t1ce_list = sorted(glob.glob(os.path.join(data_directory, '*/','*_t1ce.nii'), recursive=True))
remove1 = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_t1ce.nii'
t1ce_list = [item for item in t1ce_list if item != remove1]
flair_list = sorted(glob.glob(os.path.join(data_directory,'*/','*_flair.nii'), recursive=True))
remove2 = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_flair.nii'
flair_list = [item for item in flair_list if item != remove2]
mask_list = sorted(glob.glob(os.path.join(data_directory, '*/','*_seg.nii'), recursive=True))

# Create data generators
batch_size = 4
train_image_paths, val_image_paths, train_mask_paths, val_mask_paths = train_test_split( list(zip(t2_list, t1ce_list, flair_list)), mask_list, test_size=0.3, random_state=42)
train_image_paths, test_image_paths, train_mask_paths, test_mask_paths = train_test_split( train_image_paths, train_mask_paths, test_size=0.4, random_state=42)
print("train_image_paths =",len(train_image_paths))
print("test_image_paths =",len(test_image_paths))
print("val_image_paths =",len(val_image_paths))

train_dataset = custom_datagen(train_image_paths, train_mask_paths, batch_size)
val_dataset = custom_datagen(val_image_paths, val_mask_paths, batch_size)
test_dataset = custom_datagen(test_image_paths, test_mask_paths, batch_size)
print(train_dataset, val_dataset, test_dataset)

train_image_paths = 154
test_image_paths = 103
val_image_paths = 111
<generator object custom_datagen at 0x7ccbe04875a0> <generator object custom_datagen at 0x7ccbe0487610> <generator object custom_datagen at 0x7ccbe0487530>


In [12]:
import tensorflow as tf
def dice_coefficient(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Cast to float32
    y_pred = tf.cast(y_pred, tf.float32)  # Cast to float32
    smooth= 1e-15
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)

def dice_loss(y_true, y_pred):
    # Calculate Dice coefficient for each class and take the mean
    num_classes = 3  # Change this to the actual number of classes
    dice_coefficients = []
    for i in range(num_classes):
        class_true = y_true[..., i]
        class_pred = y_pred[..., i]
        dice_coefficients.append(dice_coefficient(class_true, class_pred))
    return 1.0 - tf.reduce_mean(dice_coefficients)


In [13]:
import tensorflow as tf

class IoUClassMetrics(tf.keras.metrics.Metric):
    def __init__(self, num_classes=4, name="iou_class", **kwargs):
        super(IoUClassMetrics, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.intersection = [self.add_weight(name=f"intersection_{i}", initializer="zeros") for i in range(num_classes)]
        self.union = [self.add_weight(name=f"union_{i}", initializer="zeros") for i in range(num_classes)]

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        for class_id in range(self.num_classes):
            intersection = tf.reduce_sum(y_true[..., class_id] * y_pred[..., class_id])
            union = tf.reduce_sum(y_true[..., class_id] + y_pred[..., class_id]) - intersection

            # Use class_id as an index to update intersection and union values
            self.intersection[class_id].assign_add(intersection)
            self.union[class_id].assign_add(union)

    def result(self):
        iou_per_class = [intersection / (union + 1e-10) for intersection, union in zip(self.intersection, self.union)]
        return iou_per_class


In [14]:
def custom_learning_rate(epoch, initial_lr, total_steps):
    if epoch < total_steps // 3:
        return initial_lr
    elif epoch < 2 * total_steps // 3:
        return initial_lr // 10
    else:
        return initial_lr // 100

In [15]:
!pip install tensorflow-addons

In [16]:
LR = 1e-4
epoch=10
import keras.optimizers as optim
from tensorflow.keras.optimizers.schedules import CosineDecay
import keras.optimizers as optim
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping
from tensorflow_addons.optimizers import AdamW
from functools import partial
#Fit the model 
model_path = '/kaggle/working/brats_slmsa.hdf5'
csv_path = '/kaggle/working/brats_slmsa.csv'

steps_per_epoch = len(train_image_paths)//batch_size
val_steps_per_epoch = len(val_image_paths)//batch_size

from tensorflow.keras.metrics import MeanIoU
from  bratsslmsa import slm_sa
num_classes = 4
iou_metric = IoUClassMetrics(num_classes=num_classes)
model = slm_sa((128,128,128,3),4)
total_steps = steps_per_epoch * epoch

lr_schedule = partial(custom_learning_rate, epoch, initial_lr=LR, total_steps=total_steps)

model.compile(optimizer = AdamW(learning_rate=lr_schedule, weight_decay=LR), loss=dice_loss, metrics=[iou_metric,dice_coefficient])
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-9, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
]
print(model.summary())

print(model.input_shape)
print(model.output_shape)

1 (None, 32, 32, 32, 32)
2 (None, 32, 32, 32, 32)
3 (None, 64, 64, 64, 32)
4 (None, 64, 64, 64, 32)
5 (None, 128, 128, 128, 32)
6 (None, 128, 128, 128, 16)
Model: "slm_sa"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d_52 (Conv3D)             (None, 64, 64, 64,   2624        ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_32 (BatchN  (No

In [17]:
# import tensorflow as tf
# from tensorflow_addons.optimizers import AdamW
# from tensorflow.keras.optimizers.schedules import CosineDecay
# LR = 1e-4
# import keras.optimizers as optim
# from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping
# model_path = '/kaggle/working/bratsslm_sa_3d.hdf5'
# csv_path = '/kaggle/working/bratsslm_sa_3d.csv'
# #Fit the model 

# steps_per_epoch = len(train_image_paths)//batch_size
# val_steps_per_epoch = len(val_image_paths)//batch_size

# from tensorflow.keras.metrics import MeanIoU
# from  bratsslmsa import slm_sa
# n_classes = 4
# IOU_keras = MeanIoU(num_classes=n_classes)
# model = slm_sa((128,128,128,3),4)
# # Define a learning rate schedule with cosine decay 
# lr_schedule = CosineDecay(initial_learning_rate=LR, decay_steps= 10)
# model.compile(optimizer=AdamW(lr_schedule), loss=dice_loss, metrics=[IOU_keras,dice_coefficient,'accuracy'])

# callbacks = [
#     ModelCheckpoint(model_path, verbose=1, save_best_only=True),
#     ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-9, verbose=1),
#     CSVLogger(csv_path),
#     EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# ]
# #model.compile(optimizer=optim.Adam(learning_rate=LR), loss=total_loss, metrics=[IOU_keras,dice_coefficient])
# print(model.summary())

# print(model.input_shape)
# print(model.output_shape)

In [18]:
history=model.fit(train_dataset,
          steps_per_epoch=steps_per_epoch,
          epochs=10,
          verbose=1,
          validation_data= val_dataset,
          validation_steps=val_steps_per_epoch,
          callbacks=callbacks)

model.save(model_path)

volume_path len: 154
Epoch 1/10
38/38 [==============================] - ETA: 0s - loss: 0.6731 - iou_class: 0.1811 - dice_coefficient: 0.5930  volume_path len: 111

Epoch 1: val_loss improved from inf to 0.77543, saving model to /kaggle/working/brats_slmsa.hdf5
38/38 [==============================] - 3242s 85s/step - loss: 0.6731 - iou_class: 0.1811 - dice_coefficient: 0.5930 - val_loss: 0.7754 - val_iou_class: 0.1203 - val_dice_coefficient: 0.4383 - lr: 1.0000e-04
Epoch 2/10
38/38 [==============================] - ETA: 0s - loss: 0.4676 - iou_class: 0.3428 - dice_coefficient: 0.9404  
Epoch 2: val_loss improved from 0.77543 to 0.69688, saving model to /kaggle/working/brats_slmsa.hdf5
38/38 [==============================] - 3214s 86s/step - loss: 0.4676 - iou_class: 0.3428 - dice_coefficient: 0.9404 - val_loss: 0.6969 - val_iou_class: 0.1988 - val_dice_coefficient: 0.7606 - lr: 1.0000e-04
Epoch 3/10
38/38 [==============================] - ETA: 0s - loss: 0.4391 - iou_class: 0.3626